## Stochastic Solution

In [1]:
from gurobipy import *
import numpy as np

In [2]:
seatNames ={0:'VIP',1:'Business Class',2:'Economy Class'}
c = [3, 2, 1] #profit from VIP, business, economy tickets sold
r = [2, 1.5, 1] # space needed of VIP, business, economy seat
d = [[25,60,200],[20,40,180],[10,25,175],[5,10,150]] #demand scenarios 

nS = len(d) #number of scenarios
p = [0.4, 0.3, 0.2, 0.1] #probability of each demand scenario

I= range(len(c)) #all seat types
S = range(nS) #all scenarios

In [3]:
#Second-stage objective coefficients
c_ForAllScen = [p[s]*c[i] for s in S for i in I]

In [4]:
m = Model("VIAStochastic")
#First stage variables - how to split seats
#x does not affect objective since no cost in partioning seats
x = m.addVars(I,lb=0,vtype=GRB.INTEGER,name='x') #x, how many of each seat type
#Second stage variables - how to sell seats
y = m.addVars(S,I,vtype=GRB.INTEGER,obj=c_ForAllScen,name='y') #y, how many of each seat type to sell

m.modelSense = GRB.MAXIMIZE

Academic license - for non-commercial use only


In [5]:
m.addConstr((sum((r[i]*x[i]) for i in I) <= 200),name='Space'); # Only have 200 spaces for seats available
m.addConstrs(((y[s,i] <= d[s][i]) for s in S for i in I ), name ='Demand'); # seats sold <= demand for seats
m.addConstrs(((y[s,i] <= x[i]) for s in S for i in I ), name='Available'); # seats sold <= seats available

In [6]:
m.optimize()
OptimalValue_VIA = m.objVal
print('\nEXPECTED PROFIT: %g' % OptimalValue_VIA)
print('STOCHASTIC SOLUTION:')
for i in I:
    print('{} {} seats made available'.format(x[i].x,seatNames[i]))
    for s in S:
        print('    Sell {} seats in scenario {}'.format(y[s,i].x, s))

Optimize a model with 25 rows, 15 columns and 39 nonzeros
Variable types: 0 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 15 rows and 3 columns
Presolve time: 0.00s
Presolved: 10 rows, 12 columns, 21 nonzeros
Found heuristic solution: objective 199.5000000
Variable types: 0 continuous, 12 integer (0 binary)

Root relaxation: objective 2.190000e+02, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  219.00000    0    4  199.50000  219.00000  9.77%     -    0s
H    0     0                     218.8000000  219.00000  0.09%     -    0s
H    0     0                     218.9000000  219.00000  0.05%     -    0s
     0     0 infeasible    0     

## Mean-Value Solution (Deterministic)

In [7]:
#Find expected demand of each seat,Ed
Ed = [0,0,0]
for i in I:
    for s in S:
        Ed[i] += d[s][i]*p[s]
print(Ed)

[18.5, 42.0, 184.0]


In [8]:
#Second-stage objective coefficients
f = []
for i in I:
    f.append(c[i])

In [9]:
m = Model("VIADeterministic")
#First stage variables - how to split seats
#x does not affect objective since no cost in partioning seats
x = m.addVars(I,lb=0,vtype=GRB.INTEGER,name='x') #x, how many of each seat type
#Second stage variables - how to sell seats
y = m.addVars(I,vtype=GRB.INTEGER,obj=f,name='y') #y, how many of each seat type to sell

m.modelSense = GRB.MAXIMIZE

In [10]:
m.addConstr((sum((r[i]*x[i]) for i in I) <= 200),name='Space'); # Only have 200 spaces for seats available
m.addConstrs(((y[i] <= Ed[i]) for i in I), name ='Demand'); # seats sold <= expected demand for seats
m.addConstrs(((y[i] <= x[i]) for i in I for s in S), name='Available'); # seats sold <= seats available

In [11]:
m.optimize()
OptimalValue_VIA = m.objVal
print('\nEXPECTED PROFIT: %g' % OptimalValue_VIA)
print('DETERMINISTIC SOLUTION:')
for i in I:
    print('{} {} seats made available'.format(x[i].x,seatNames[i]))
    for s in S:
        print('    Sell {} seats'.format(y[i].x))

Optimize a model with 16 rows, 6 columns and 30 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 15 rows and 3 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Found heuristic solution: objective 239.0000000
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 12 (of 12 available processors)

Solution count 2: 239 101 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.390000000000e+02, best bound 2.390000000000e+02, gap 0.0000%

EXPECTED PROFIT: 239
DETERMINISTIC SOLUTION:
18.0 VIP seats made available
    Sell 18.0 seats
    Sell 18.0 seats
    Sell 18.0 seats
    Sell 18.0 seats
42.0 Business Class

In [12]:
xMV = [18.0, 42.0, 101.0] #Mean-value problem solution

## VSS
### Fix x = xMV in the stochastic model

In [13]:
m = Model("VIA-VSS")
#First stage variables - how to split seats
#x does not affect objective since no cost in partioning seats
x = m.addVars(I,lb=0,vtype=GRB.INTEGER,name='x') #x, how many of each seat type
#Second stage variables - how to sell seats
y = m.addVars(S,I,vtype=GRB.INTEGER,obj=c_ForAllScen,name='y') #y, how many of each seat type to sell

m.modelSense = GRB.MAXIMIZE

In [14]:
m.addConstr((sum((r[i]*x[i]) for i in I) <= 200),name='Space'); # Only have 200 spaces for seats available
m.addConstrs(((x[i] == xMV[i]) for i in I), name='xMV')# Fix x to xMV
m.addConstrs(((y[s,i] <= d[s][i]) for i in I for s in S), name ='Demand'); # seats sold <= demand for seats
m.addConstrs(((y[s,i] <= x[i]) for i in I for s in S), name='Available'); # seats sold <= seats available


In [15]:
m.optimize()
OptimalValue_VIA = m.objVal
print('\nEXPECTED PROFIT WITH x = xMV: %g' % OptimalValue_VIA)
print('STOCHASTIC SOLUTION WITH x = xMV:')
for i in I:
    print('{} {} seats made available'.format(x[i].x,seatNames[i]))
    for s in S:
        print('    Sell {} seats in scenario {}'.format(y[s,i].x, s))

Optimize a model with 28 rows, 15 columns and 42 nonzeros
Variable types: 0 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+02]
Found heuristic solution: objective 215.9000000
Presolve removed 28 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 215.9 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.159000000000e+02, best bound 2.159000000000e+02, gap 0.0000%

EXPECTED PROFIT WITH x = xMV: 215.9
STOCHASTIC SOLUTION WITH x = xMV:
18.0 VIP seats made available
    Sell 18.0 seats in scenario 0
    Sell 18.0 seats in scenario 1
    Sell 10.0 seats in scenario 2
    Sell 5.0 seats in scenario 3
42.0 Business Class seats made available
    Sell 42.0 seats in scenario 0
    Sell 40.0 sea

### VSS = 218.9P - 215.9P = 3P, where P is the price of an economy class ticket.

## EVPI 
### Determine wait-and-see solutions for each scenario

In [16]:
def waitAndSeeProfitPerScenario(dScenario,scenarioIndex):
    #Second-stage objective coefficients    
    m = Model("VIA-Scenario")
    #First stage variables - how to split seats
    #x does not affect objective since no cost in partioning seats
    x = m.addVars(I,lb=0,vtype=GRB.INTEGER,name='x') #x, how many of each seat type
    #Second stage variables - how to sell seats
    y = m.addVars(I,vtype=GRB.INTEGER,obj=f,name='y') #y, how many of each seat type to sell
    
    m.addConstr((sum((r[i]*x[i]) for i in I) <= 200),name='Space'); # Only have 200 spaces for seats available
    m.addConstrs(((y[i] <= dScenario[i]) for i in I), name ='Demand'); # seats sold <= expected demand for seats
    m.addConstrs(((y[i] <= x[i]) for i in I for s in S), name='Available'); # seats sold <= seats available

    m.modelSense = GRB.MAXIMIZE
    m.optimize()
    OptimalValue_VIA = m.objVal
    print('\nEXPECTED PROFIT: %g' % OptimalValue_VIA)
    print('Scenario {} SOLUTION:'.format(scenarioIndex))
    for i in I:
        print('{} {} seats made available'.format(x[i].x,seatNames[i]))
        for s in S:
            print('    Sell {} seats'.format(y[i].x))

In [17]:
for dScenario in d:
    waitAndSeeProfitPerScenario(dScenario, d.index(dScenario)+1)

Optimize a model with 16 rows, 6 columns and 30 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 15 rows and 3 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Found heuristic solution: objective 201.0000000
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: objective 2.550000e+02, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     255.0000000  255.00000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.02 seconds
Thread count was 12 (of 12 available processors)

Solution count 3: 255 201 60 

Optimal solution found (tolerance 1.00e-04)
Best ob

In [18]:
waitAndSeeScenarioProfit = [255,240,222,185]
EVwaitAndSee = np.array(waitAndSeeScenarioProfit) * np.array(p)
np.sum(EVwaitAndSee)

236.9

### EVPI = 236.9P - 218.9P = 18P, where P is the price of an economy class ticket.